# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fcb68cda9d0>
├── 'a' --> tensor([[-1.1359, -1.6183,  0.6523],
│                   [-0.3067,  0.4747,  2.1281]])
└── 'x' --> <FastTreeValue 0x7fcb68cdac70>
    └── 'c' --> tensor([[ 1.1237,  0.3441,  0.3296,  0.2967],
                        [-0.1333,  1.4504,  0.1709,  0.1092],
                        [-0.2046, -0.7908, -1.4531,  0.1742]])

In [4]:
t.a

tensor([[-1.1359, -1.6183,  0.6523],
        [-0.3067,  0.4747,  2.1281]])

In [5]:
%timeit t.a

60.2 ns ± 0.0664 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fcb68cda9d0>
├── 'a' --> tensor([[ 0.3623, -0.9445, -1.2747],
│                   [ 1.6428, -0.5569,  0.1611]])
└── 'x' --> <FastTreeValue 0x7fcb68cdac70>
    └── 'c' --> tensor([[ 1.1237,  0.3441,  0.3296,  0.2967],
                        [-0.1333,  1.4504,  0.1709,  0.1092],
                        [-0.2046, -0.7908, -1.4531,  0.1742]])

In [7]:
%timeit t.a = new_value

62.7 ns ± 0.02 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-1.1359, -1.6183,  0.6523],
               [-0.3067,  0.4747,  2.1281]]),
    x: Batch(
           c: tensor([[ 1.1237,  0.3441,  0.3296,  0.2967],
                      [-0.1333,  1.4504,  0.1709,  0.1092],
                      [-0.2046, -0.7908, -1.4531,  0.1742]]),
       ),
)

In [10]:
b.a

tensor([[-1.1359, -1.6183,  0.6523],
        [-0.3067,  0.4747,  2.1281]])

In [11]:
%timeit b.a

54 ns ± 0.0727 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 1.0520,  0.9907, -0.8478],
               [-0.8270,  0.7219,  1.0576]]),
    x: Batch(
           c: tensor([[ 1.1237,  0.3441,  0.3296,  0.2967],
                      [-0.1333,  1.4504,  0.1709,  0.1092],
                      [-0.2046, -0.7908, -1.4531,  0.1742]]),
       ),
)

In [13]:
%timeit b.a = new_value

490 ns ± 0.145 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

835 ns ± 0.149 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.4 µs ± 21.3 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

139 µs ± 548 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

138 µs ± 976 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fcab25cf460>
├── 'a' --> tensor([[[-1.1359, -1.6183,  0.6523],
│                    [-0.3067,  0.4747,  2.1281]],
│           
│                   [[-1.1359, -1.6183,  0.6523],
│                    [-0.3067,  0.4747,  2.1281]],
│           
│                   [[-1.1359, -1.6183,  0.6523],
│                    [-0.3067,  0.4747,  2.1281]],
│           
│                   [[-1.1359, -1.6183,  0.6523],
│                    [-0.3067,  0.4747,  2.1281]],
│           
│                   [[-1.1359, -1.6183,  0.6523],
│                    [-0.3067,  0.4747,  2.1281]],
│           
│                   [[-1.1359, -1.6183,  0.6523],
│                    [-0.3067,  0.4747,  2.1281]],
│           
│                   [[-1.1359, -1.6183,  0.6523],
│                    [-0.3067,  0.4747,  2.1281]],
│           
│                   [[-1.1359, -1.6183,  0.6523],
│                    [-0.3067,  0.4747,  2.1281]]])
└── 'x' --> <FastTreeValue 0x7fcab1f84b20>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.4 µs ± 32.5 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fcab8bdf6d0>
├── 'a' --> tensor([[-1.1359, -1.6183,  0.6523],
│                   [-0.3067,  0.4747,  2.1281],
│                   [-1.1359, -1.6183,  0.6523],
│                   [-0.3067,  0.4747,  2.1281],
│                   [-1.1359, -1.6183,  0.6523],
│                   [-0.3067,  0.4747,  2.1281],
│                   [-1.1359, -1.6183,  0.6523],
│                   [-0.3067,  0.4747,  2.1281],
│                   [-1.1359, -1.6183,  0.6523],
│                   [-0.3067,  0.4747,  2.1281],
│                   [-1.1359, -1.6183,  0.6523],
│                   [-0.3067,  0.4747,  2.1281],
│                   [-1.1359, -1.6183,  0.6523],
│                   [-0.3067,  0.4747,  2.1281],
│                   [-1.1359, -1.6183,  0.6523],
│                   [-0.3067,  0.4747,  2.1281]])
└── 'x' --> <FastTreeValue 0x7fcab8bdf3d0>
    └── 'c' --> tensor([[ 1.1237,  0.3441,  0.3296,  0.2967],
                        [-0.1333,  1.4504,  0.1709,  0.1092],
                 

In [23]:
%timeit t_cat(trees)

30.5 µs ± 61.2 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

59 µs ± 47.1 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 1.1237,  0.3441,  0.3296,  0.2967],
                       [-0.1333,  1.4504,  0.1709,  0.1092],
                       [-0.2046, -0.7908, -1.4531,  0.1742]],
              
                      [[ 1.1237,  0.3441,  0.3296,  0.2967],
                       [-0.1333,  1.4504,  0.1709,  0.1092],
                       [-0.2046, -0.7908, -1.4531,  0.1742]],
              
                      [[ 1.1237,  0.3441,  0.3296,  0.2967],
                       [-0.1333,  1.4504,  0.1709,  0.1092],
                       [-0.2046, -0.7908, -1.4531,  0.1742]],
              
                      [[ 1.1237,  0.3441,  0.3296,  0.2967],
                       [-0.1333,  1.4504,  0.1709,  0.1092],
                       [-0.2046, -0.7908, -1.4531,  0.1742]],
              
                      [[ 1.1237,  0.3441,  0.3296,  0.2967],
                       [-0.1333,  1.4504,  0.1709,  0.1092],
                       [-0.2046, -0.7908, -1.4531,  0.1742]],

In [26]:
%timeit Batch.stack(batches)

77.7 µs ± 146 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 1.1237,  0.3441,  0.3296,  0.2967],
                      [-0.1333,  1.4504,  0.1709,  0.1092],
                      [-0.2046, -0.7908, -1.4531,  0.1742],
                      [ 1.1237,  0.3441,  0.3296,  0.2967],
                      [-0.1333,  1.4504,  0.1709,  0.1092],
                      [-0.2046, -0.7908, -1.4531,  0.1742],
                      [ 1.1237,  0.3441,  0.3296,  0.2967],
                      [-0.1333,  1.4504,  0.1709,  0.1092],
                      [-0.2046, -0.7908, -1.4531,  0.1742],
                      [ 1.1237,  0.3441,  0.3296,  0.2967],
                      [-0.1333,  1.4504,  0.1709,  0.1092],
                      [-0.2046, -0.7908, -1.4531,  0.1742],
                      [ 1.1237,  0.3441,  0.3296,  0.2967],
                      [-0.1333,  1.4504,  0.1709,  0.1092],
                      [-0.2046, -0.7908, -1.4531,  0.1742],
                      [ 1.1237,  0.3441,  0.3296,  0.2967],
                   

In [28]:
%timeit Batch.cat(batches)

144 µs ± 285 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

315 µs ± 1.9 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
